In [38]:
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder  
import pandas as pd
import numpy as np
DEBUG = 0
encoder = LabelEncoder()
data_watermelon = read_csv('watermelon3_0_En.csv')
data_watermelon.head()

,No.,Color,Root,Knocks,Texture,Umbilicus,Touch,Density,SugerRatio,Label
0,1,green,curl,heavily,distinct,sunken,hard,0.697,0.460,1
1,2,black,curl,dull,distinct,sunken,hard,0.774,0.376,1
2,3,black,curl,heavily,distinct,sunken,hard,0.634,0.264,1
3,4,green,curl,dull,distinct,sunken,hard,0.608,0.318,1
4,5,white,curl,heavily,distinct,sunken,hard,0.556,0.215,1


In [2]:
data_watermelon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 10 columns):
No.           17 non-null int64
Color         17 non-null object
Root          17 non-null object
Knocks        17 non-null object
Texture       17 non-null object
Umbilicus     17 non-null object
Touch         17 non-null object
Density       17 non-null float64
SugerRatio    17 non-null float64
Label         17 non-null int64
dtypes: float64(2), int64(2), object(6)
memory usage: 1.4+ KB


In [27]:
data_watermelon.drop(['No.'], axis=1, inplace=True)
X0 = data_watermelon.drop(['Label'], axis = 1)
y = data_watermelon['Label'].copy()
# X_test = X0[0:1]  #把第一个样本作为测试集
# y_test = y[0:1]
# X0 = X0[1:]
# y = y[1:]
# print(len(X0))

In [18]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
text_feature = list(X0)[:-2]
continuous_feature = list(X0)[-2:]
encoder.fit_transform(X0[text_feature[1]]) #注意这里映射之后的结果为列向量，并不能直接用dataframe转换
# encoder.transform(X_test[text_feature[2]])
encoder.transform(['curl'])

array([0], dtype=int64)

In [6]:
X_text_features = [encoder.fit_transform(X0[feature]) for feature in text_feature]
print(X_text_features)
import pandas as pd
import numpy as np
X = pd.DataFrame(np.c_[np.array(X_text_features).T, np.array(X0[['Density','SugerRatio']])],columns=list(X0))
print(X[:6])

[array([1, 0, 0, 1, 2, 1, 0, 0, 0, 1, 2, 2, 1, 2, 0, 2, 1], dtype=int64), array([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 0, 1, 1, 1, 0, 0], dtype=int64), array([2, 1, 2, 1, 2, 2, 2, 2, 1, 0, 0, 2, 2, 1, 2, 2, 1], dtype=int64), array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 2, 0, 0, 1, 2, 0], dtype=int64), array([2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1, 1, 3, 3, 0, 1, 0], dtype=int64), array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0], dtype=int64)]
   Color  Root  Knocks  Texture  Umbilicus  Touch  Density  SugerRatio
0    1.0   0.0     2.0      1.0        2.0    0.0    0.697       0.460
1    0.0   0.0     1.0      1.0        3.0    0.0    0.774       0.376
2    0.0   0.0     2.0      1.0        3.0    0.0    0.634       0.264
3    1.0   0.0     1.0      1.0        3.0    0.0    0.608       0.318
4    2.0   0.0     2.0      1.0        3.0    0.0    0.556       0.215
5    1.0   1.0     2.0      1.0        0.0    1.0    0.403       0.237


In [7]:
[name for name in list(X0) if name not in text_feature]

['Density', 'SugerRatio']

In [28]:
#注意：到时候对于测试集，也要经过上述一系列预处理。故考虑写一个custom transformer
from sklearn.base import BaseEstimator, TransformerMixin
class Melon_attribute_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, text_features):
        self.text_features = text_features
    def fit(self, X, y = None):
        self.continuous_features = [name for name in list(X) if name not in text_feature]
        return self # 不做处理
    def transform(self, X, y = None):
        X_text_features = [encoder.fit_transform(X[feature]) for feature in self.text_features]
        # 这样写虽然能解决X_text_features的映射问题，但是实际上使用了六个enconder，最终encoder的结果是由最后一个feature所fit而得的
        # 所以无法利用前五个encoder对前五种feature进行transform
        # 故：encoder.transform(X_test) 结果出错，显示出现了新的feature
        return pd.DataFrame(np.c_[np.array(X_text_features).T, np.array(X[self.continuous_features])],columns=list(X))

text_feature = list(X0)[:-2]
melon_transformer = Melon_attribute_transformer(text_features=text_feature)
X_transformed = melon_transformer.fit_transform(X0, y)
# melon_transformer.predict(X_test)
X_test = np.array(X_transformed[1:2])[0] #注意：在函数中若使用切片，则操作np.array比dataframe方便
y_test = np.array(y[1:2])
X = np.array(X_transformed[2:])
y = np.array(y[2:])

In [35]:
y[3]

1

In [37]:
def naive_bayesian(X, y, predict):
    #     predict = melon_transformer.transform(predict)  #没有能够实现用自己定义的transformer对原始test数据也进行转换....
    label_unique = set(y)
    num_labels = len(label_unique)
    joint_p = num_labels * [1]  # [1, 1]  joint_p 代表对于每个类别C，求出他的若干个相互独立的p(x_i|C)的乘积，即联合概率密度
    for label_index, label in enumerate(label_unique):
        #         X_C = X[y == label]  ##这里由于y是向量不是矩阵，所以索引不方便
        X_C = X[np.array(y == label).T]
        P_C = (len(X_C) + 1) / (len(y) + num_labels )
        for feature_index, x_i in enumerate(predict):
            num_features = len(set(X_C[:, feature_index]))
            if feature_index in range(6, 8):  # 第六、七个属性为连续属性
                mean_c_xi = np.mean(X_C[:, feature_index])  # 求出x_i 的均值和标准差以估计概率密度函数
                std_c_xi = np.std(X_C[:, feature_index],ddof=1) #为了和书中保持结果严格一致，使用无偏标准差计算
                joint_p[label_index] *= np.exp(-(x_i - mean_c_xi) ** 2 / (2 * std_c_xi ** 2)) / np.sqrt(
                    np.pi * 2) / std_c_xi
            else:  #错误：忘记加else，导致最后两个连续属性计算完之后又接着乘以下面的D_c_xi
                D_C_xi = np.sum(X_C[:, feature_index] == x_i)
                joint_p[label_index] *= (D_C_xi + 1) / (len(X_C) + num_features )
        joint_p[label_index] *= P_C
    max_p = 0
    best_ind = 0
    for ind, prob in enumerate(joint_p):  #未加入拉普拉斯修正时：第一个样本结果：坏瓜：6.85*10-5  第二个样本结果： 0.0523
        if max_p < prob:
            max_p = prob
            best_ind = ind
    return best_ind

if DEBUG:
    print('预测结果为：',naive_bayesian(X, y, X[10]), '实际结果为：', y[10])
else:
    print('预测结果为：',naive_bayesian(X, y, X_test), '实际结果为：', y_test)
#实验结果：所有的训练集结果均严格正确。西瓜书152页p凹陷|是 有错，应为5/8         
                

NameError: name 'DEBUG' is not defined

In [49]:
    label_unique = set(y)
    num_labels = len(label_unique)
    joint_p = num_labels*[1] # [1, 1]
    